# Comparitive study of before and after flash flood in Cologne using NDWI service available in EOplaza

In this notebook, we tried in performing comparative study between pre and post image for Colong during 2021 flood. A simple technique to subtract pre and post image is done to know the change in water content due to flood in that region. Refernce: https://labo.obs-mip.fr/multitemp/the-ndwi-applied-to-the-recent-flooding-in-the-central-us/

In [1]:
# import necessary packages
import openeo
from openeo.api.process import Parameter
import json
from pathlib import Path
import matplotlib.pyplot as plt
import rasterio
import numpy as np

# connect with the backend
eoconn = openeo.connect("openeo.vito.be").authenticate_oidc()

Authenticated using refresh token.


User can choose among different backend available [here](https://hub.openeo.org/) to connect to the backend. Rrgarding the authentication process OpenID connect (oidc) is recommended, but not always straightforward to use. In cases where you are unable to connect with the backend use basic authentication method explained [here](https://openeo.org/documentation/1.0/authentication.html#openid-connect).

In [2]:
# function to load geojson file
def read_json(path: Path) -> dict:
    with open(path) as input:
        field = json.load(input)
        input.close()
    return field

Since this is an already published service available service, they need not be concerned with selecting the backend. They can directly execute the process by providing time and area of interest.

In [3]:
before_date = ["2021-05-12","2021-06-12"]
after_date = ["2021-06-18", "2021-07-18"]
aoi = read_json("/home/pratixa/Private/aoi/cologne_all.geojson")

# Create a processing graph from the NDWI process using an active openEO connection
before_ndwi = eoconn.datacube_from_process("NDWI", namespace="vito", date=before_date
                                        ,polygon=aoi)
# Create a processing graph from the NDWI process using an active openEO connection
after_ndwi = eoconn.datacube_from_process("NDWI", namespace="vito", date=after_date
                                        ,polygon=aoi)

/home/pratixa/.local/lib/python3.6/site-packages/openeo/metadata.py:252: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")


As you can see a userwarning for missing data pops up which might not be an issue in normal case but here we wish to further evaluate our result in our process thus defining metadata is needed.

Not all the available service requires updating metadata. If a service lack metadata then performing further computation on output of the service could be an issue. In such case user can update the metadata based on it's status.

In [5]:
# check available information is available in metadata or not
before_ndwi.metadata

In [6]:
# updating our metadata
from openeo.metadata import CollectionMetadata

before_ndwi.metadata = CollectionMetadata({"cube:dimensions":{"t":{"type":"temporal"}}})
after_ndwi.metadata = CollectionMetadata({"cube:dimensions":{"t":{"type":"temporal"}}})

Once the metadata is updated you can perform further operations like subtraction or sum etc as done for this use case.

Since now we have details on temporal dimension we can perform dimension reduction. As we loaded our collection for specific time intervals, it can include multiple datacubes. Thus [reduce_dimension](https://processes.openeo.org/#reduce_dimension) applies a reducer to a data cube dimension by collapsing all the pixel values along the time dimension into an output value computed by the reducer. 

It is then followed by subtracting our before datacube from the later.

In [7]:
# compute the change between pre and post image
merging_cubes = after_ndwi.merge_cubes(-before_ndwi)
differenced_cube = merging_cubes.reduce_dimension(dimension="t",reducer='sum')

Once the process is completed, you can also save it as your process using [save_user_defined_process](https://open-eo.github.io/openeo-python-client/udp.html) that can later be used for a similar task. Otherwise, you can download the result either by direct download (in case of the small spatial extent with few processing) or perform create a [batch job](https://open-eo.github.io/openeo-python-client/batch_jobs.html) in case it is a heavy task over a large extent.

In [9]:
# download your result either using synchronous method or batch
# synchronous download
differenced_cube.download("changed_ndwi.tiff")
# 
# # Or perform batch processing if area is comparatively large
# batch_job = Rrescaled_chunks.create_job(out_format = "GTiff", title="changed_ndwi")
# batch_job.start_and_wait()
# results = batch_job.get_results()
# results.download_files()